In [1]:
def generate_name_classification_prompt(input: str, categories: list, rules: dict) -> str:
    """
    Generates a classification prompt for OpenAI with instructions to return JSON output.
    """
    rule_text = ""
    for cat in categories:
        rule_text += f"\n**{cat} Rules:**\n- " + "\n- ".join(rules[cat]) + "\n"

    prompt = f"""You are a classification assistant. Your task is to classify a given text input into one of the following categories based on custom rules.

**text input:** {input}

**Categories:** {", ".join(categories)}

{rule_text}

Instructions:
- Analyze the text input using the rules above.
- Decide the most appropriate category according to rules.
- Don't use any predefined rules or dont make any assumptions
- Strictly use only the rules given to you
- if there are any words that overlaps the category then fully get dependent on the rules.
- Return only a valid JSON in the format: {{ "object_location": "<Final Category>" }}
"""
    return prompt


In [15]:
import openai
import json
from openai import OpenAI
from google.colab import userdata
client = OpenAI(api_key=userdata.get('OPEN_AI_KEY'))

categories = ["schoolname", "region", "charter"]

rules = {
    "schoolname": [
        "The schoolname is usually contains the names of some entities or some society or there will be term school or intitute at the end of the name given",
        "if there is word school in the text, then force fully categorize it to school ,even when there are other terms in the text",
        ""
        "Common terms: studentid, studentname, credits, no_day_active,classs,schoolactivities,school,School"
    ],
    "region": [
        "it refers to the area or location followed by the explicitly given 'region' at the end of the given name",
        "Only classify to this category if there is explicitly mentioned region ",
        "if there is not given 'region' word in the name consider the other category",
        "if there is term 'region' but there is also the word school then give priority to school ",
        "Common terms: region"
    ],
    "charter": [
        "the charter category is explicitly defined in the name and at the end or in some place there will be the term 'charter'",
        "if there is not mention term 'charter' then consider any other category but don't use charter category",
        "if 'charter' is the and the other category term is there then don't classify to charter,make sure there is only the charter term in the name , not any other category terms"
        "Common terms: charter"
    ]
}

# Example name to classify
name_to_classify = "How many childrens are from Desert Sands Charter High School?"

# Generate prompt
prompt = generate_name_classification_prompt(name_to_classify, categories, rules)

# Query model
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": prompt}],
    temperature=0,
    max_tokens=150

)

# Extract and parse JSON response
import json

output_text = response.choices[0].message.content.strip()

# Print the raw output to debug
print("RAW OUTPUT FROM OPENAI:\n", output_text)

try:
    result_json = json.loads(output_text)
    print("Parsed JSON:", result_json)
except json.JSONDecodeError as e:
    print("JSON parsing failed:", e)


RAW OUTPUT FROM OPENAI:
 ```json
{ "object_location": "schoolname" }
```
JSON parsing failed: Expecting value: line 1 column 1 (char 0)
